In diesem Notebook möchte ich die autograd funktion von pytorch anhand eines kleinen Netzwerkes testen.

In [38]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd.functional import jacobian
from torch.autograd import grad

Erstellung des einfachen Netzwerkes

In [39]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Schichten festlegen
        self.Layer_1 = nn.Linear(3, 16)
        self.Output_Layer = nn.Linear(16, 2)

        # Aktivierungsfunktion definieren
        self.activation_fnc = nn.ReLU()

    def forward(self, x):
        x = self.Layer_1(x)
        x = self.activation_fnc(x)
        x = self.Output_Layer(x)

        return x

test_network = SimpleNN()

Ein paar einfache werte austesten (mit Gradientenverfolgung des Eingangs)

In [40]:
test_input = torch.tensor([[1, 2, 1.5], [1.3, 2.5, 1.1]], dtype=torch.float32, requires_grad=True)
print('Test Input: \n', test_input)

test_output = test_network(test_input)
print('Output: \n', test_output)

Test Input: 
 tensor([[1.0000, 2.0000, 1.5000],
        [1.3000, 2.5000, 1.1000]], requires_grad=True)
Output: 
 tensor([[0.7466, 0.3899],
        [0.8794, 0.4927]], grad_fn=<AddmmBackward0>)


Gradienten bezüglich der Eingangsgrößen berechnen (man kann den Gradienten nur bezüglich einer skalaren Ausgangsgröße berechnen)

In [ ]:
J = jacobian(lambda inp: test_network(inp), test_input[0], create_graph=True)  # lambda ist in diesem fall einfach eine kleine Funktion (nur nicht über def), da jacobian eine Funktion als eingabe erwartet. Die Funktion wird dann mit dem test_input ausgewertet und der Gradient wird erstellt.
print('Jacobimatrix berechnen: \n', J)

Jacobimatrix berechnen: 
 tensor([[-0.0858,  0.2688,  0.0785],
        [ 0.1003,  0.0643, -0.0129]], grad_fn=<ViewBackward0>)


In [44]:
gradient_1 = grad(test_output[:, 0].sum(), test_input, create_graph=True)
gradient_2 = grad(test_output[:, 1].sum(), test_input, create_graph=True)

print('Berechnung Gradient 1 mit Grad: \n', gradient_1)
print('Berechnung Gradient 2 mit Grad: \n', gradient_2)

Berechnung Gradient 1 mit Grad: 
 (tensor([[-0.0858,  0.2688,  0.0785],
        [ 0.0272,  0.3049, -0.0486]], grad_fn=<MmBackward0>),)
Berechnung Gradient 2 mit Grad: 
 (tensor([[ 0.1003,  0.0643, -0.0129],
        [ 0.1722,  0.0873, -0.0938]], grad_fn=<MmBackward0>),)


Prüfen, ob die backward() funktion auch auf Basis der Jacobimatrix funktioniert.

In [43]:
test_loss = J.sum()
print('Test Loss: \n', test_loss)
print()

test_loss.backward()
print('Gradient des Test Loss nach eingängen: \n', test_input.grad)
print()

name_param = list(test_network.named_parameters())
print(name_param[0][0], ': \n', name_param[0][1].grad)
print()
print(name_param[1][0], ': \n', name_param[1][1].grad)


Test Loss: 
 tensor(0.4133, grad_fn=<SumBackward0>)

Gradient des Test Loss nach eingängen: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])

Layer_1.weight : 
 tensor([[ 0.2019,  0.2019,  0.2019],
        [-0.1288, -0.1288, -0.1288],
        [-0.0281, -0.0281, -0.0281],
        [ 0.1557,  0.1557,  0.1557],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.2105,  0.2105,  0.2105],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.1123,  0.1123,  0.1123],
        [ 0.1484,  0.1484,  0.1484],
        [-0.2207, -0.2207, -0.2207],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.2296,  0.2296,  0.2296],
        [ 0.3434,  0.3434,  0.3434]])

Layer_1.bias : 
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Anmerkung: Wenn keine aktivierungsfunktion verwendet wird, passiert folgendes:

-Gradient des Test Loss bezüglich der Eingänge ist None
    
-Gradient des bias bezüglich der Eingänge ist None

Das liegt daran, dass diese Einträge beim Ableiten herausfliegen. (hiernach fehlt etwas - siehe chatgpt oder andere skripte)